In [1]:
import numpy as np
import pandas as pd 
import math
import warnings
warnings.filterwarnings('ignore')

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, ClassificationPreset
from evidently.test_suite import TestSuite
from evidently.test_preset import BinaryClassificationTestPreset, DataStabilityTestPreset
from evidently.options import ColorOptions

color_scheme = ColorOptions(
    primary_color = "#5a86ad",
    fill_color = "#fff4f2",
    zero_line_color = "#016795",
    current_data_color = "#c292a1",
    reference_data_color = "#017b92"
)

path1 = "/Users/pooja/Desktop/GitHub/mrdc_model_monitoring/data/"
path2 = "/Users/pooja/Desktop/GitHub/mrdc_model_monitoring/reports/"

In [2]:
features = ['ach_c_median_past30d', 
            'ach_c_std_past30d',
            'ach_d_avg_past10d',
            'ach_d_median_past30d', 
            'amount', 
            'avg_running_balance_past30d',
            'card_txn_std_past10by30d', 
            'card_txn_std_past10d',
            'credit_txn_avg_past10by30d', 
            'credit_txn_avg_past10d',
            'credit_txn_count_past10by30d', 
            'credit_txn_std_past30d',
            'debit_txn_count_past10by30d', 
            'ein_ssn', 
            'is_between1000and5000',
            'is_between200and1000', 
            'mrdc_c_avg_past30d',
            'mrdc_c_median_past10by30d', 
            'od_count_past30d',
            'past10by30d_between200and1000_ratio', 
            'past10by30d_check_ratio',
            'past10by30d_returned_check_ratio', 
            'pd_avg_past10d',
            'rejected_past10by30d_between200and1000_ratio',
            'returned_past30d_avg_check_amount', 
            'rn_past30d_avg_check_amount',
            'past10by30d_lessthan200_ratio']

drift_cols = ['ach_d_avg_past10d', 'card_txn_std_past10by30d', 'card_txn_std_past10d', 'credit_txn_avg_past10by30d',
               'credit_txn_avg_past10d', 'credit_txn_count_past10by30d', 'debit_txn_count_past10by30d', 'mrdc_c_median_past10by30d',
                'pd_avg_past10d' ]





In [3]:
# reference dataset 
df_ref = pd.read_csv(path1 + 'mrdc_training.csv')
df_ref.columns = df_ref.columns.str.lower()
df_ref.shape

(371825, 32)

In [4]:
# current dataset 
def load_dataset(month):
    df = pd.read_csv(path1 + "monthly/"+ month + ".csv")
    df.columns = df.columns.str.lower()
    return df

In [5]:
df_ref[:2]

,mrdc_id,created_status,total_amount,predictions,status,ach_c_median_past30d,ach_c_std_past30d,ach_d_avg_past10d,ach_d_median_past30d,amount,...,mrdc_c_median_past10by30d,od_count_past30d,past10by30d_between200and1000_ratio,past10by30d_check_ratio,past10by30d_returned_check_ratio,pd_avg_past10d,rejected_past10by30d_between200and1000_ratio,returned_past30d_avg_check_amount,rn_past30d_avg_check_amount,past10by30d_lessthan200_ratio
0,2cdf193d-4844-4eb6-891b-80f28f39c682,2022-01-01,15000,944.605947,deposited,0.0,0.0,0.0,0.0,150.0,...,0.0,0,0.0,0.2,0.0,0.0,0.0,0.0,75.0,0.2
1,77a413c3-e7b8-4d98-9e3e-38cafd052870,2022-01-01,50000,804.125965,deposited,0.0,0.0,0.0,0.0,500.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_ref['status'].value_counts()

status
deposited            332075
rejectPreTransmit     30880
rejected               4564
returned               4306
Name: count, dtype: int64

In [7]:
# considering returned checks 
df_ref =  df_ref[(df_ref['status'] == 'returned') | (df_ref['status'] == 'deposited')]
df_ref = df_ref[features + ['predictions', 'status']]
df_ref['actual_return'] = df_ref['status'].apply(lambda x: 0 if x == 'deposited' else 1 )
df_ref['probability'] = df_ref['predictions'] / 1000


In [8]:
def preprocess(df): 
    df = df[features + ['score' , 'status']]
    df = df[(df['status'] == 'returned') | (df['status'] == 'deposited')]
    df.rename(columns={'score': 'predictions'}, inplace = True)
    df['actual_return'] = df['status'].apply(lambda x: 0 if x == 'deposited' else 1 )
    df['probability'] = df['predictions'] / 1000 
    return df

In [60]:
# ks table 

def ks_table(target, prob, month):
    data = pd.DataFrame()
    data['y'] = target
    data['y'] = data['y'].astype(float)
    data['p'] = prob
    data['y0'] = 1- data['y']
    data['bucket'] = pd.cut(data['p'], bins = [0.0,0.100, 0.200, 0.300, 0.500, 0.700, 0.800, 0.850,0.900,0.950,1.000] )
    grouped = data.groupby('bucket', as_index=True)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()['p']
    kstable['max_prob'] = grouped.max()['p']
    kstable['events'] = grouped.sum()['y']
    kstable['nonevents'] = grouped.sum()['y0']
    kstable = kstable.sort_values(by='min_prob', ascending=False).reset_index(drop=True)
    kstable['event_rate'] = (kstable.events / data['y'].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable['nonevents'] /  data['y0'].sum()).apply('{0:2%}'.format)
    kstable['cum_eventrate'] = (kstable.events / data['y'].sum()).cumsum()
    kstable['cum_noneventrate'] = (kstable.nonevents / data['y0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate'] - kstable['cum_noneventrate'], 3) * 100
    kstable['bad_rate'] = (kstable['events'] / (kstable['events'] + kstable['nonevents'])) * 100
    average_event_rate=kstable['events'].sum()/(kstable['events'].sum()+kstable['nonevents'].sum())
    kstable['Lift']=np.round((kstable['bad_rate']/average_event_rate)/100,2)
    # formatting
    kstable['cum_eventrate'] = kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate'] = kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Probability', inplace=True)
    pd.set_option('display.max_columns', 9)
    # kstable.to_csv(path2 + "mrdc_model_drift/kstable/kstable_" + month + ".csv")
    # print(kstable,"\n\n")
    return str(round(max(kstable['KS']),3)), str((kstable.index[kstable['KS']==max(kstable['KS'])][0]))

In [61]:
months = ['sept_23', 'oct_23', 'nov_23', 'dec_23', 'jan_24']
for i in range(len(months)): 
    df = load_dataset(months[i])
    df = preprocess(df)
    ks_value = ks_table(df['actual_return'], df['probability'], months[i])
    print(ks_value)

('46.400000000000006', '8')
('50.7', '7')
('33.4', '4')
('37.3', '4')
('25.5', '4')


In [11]:
column_mapping = ColumnMapping()
column_mapping.target = 'actual_return'
column_mapping.prediction = 'probability'
column_mapping.numerical_features = features

In [12]:
def label_binary_classification():
        label_binary_classification_performance = TestSuite(tests=[
            BinaryClassificationTestPreset(stattest='psi'),
        ],options=[color_scheme])

        label_binary_classification_performance.run(reference_data = oct, current_data=df_ref, column_mapping = column_mapping)
        label_binary_classification_performance.save_html(path2 + "mrdc_model_drift/label_classification_oct_23.html")
        return label_binary_classification_performance

In [13]:
# report  = label_binary_classification()
# report.show(mode='inline')

##### PSI value (target value) 

In [14]:

def calculate_psi(df_ref, df_cur, features, month): 
    name = features
    score_file_old = df_ref.copy()
    score_file_new = df_cur.copy()
    psi_values = []
    bins = [0, 100, 200, 300, 500, 700, 800, 850, 900, 950, 1000]
    
    for m in range(len(name)):
        pred_score = pd.DataFrame()
        pred_score["y_predict"] = score_file_old[name[m]]

        pred_score_new = pd.DataFrame()
        pred_score_new["y_predict"] = score_file_new[name[m]]
        pred_score_new = pred_score_new.fillna(-99999)
        pred_score_new = pred_score_new[(pred_score_new.y_predict != -99999)]

        obs_new_vs_old = pd.DataFrame(data=np.NaN, index=range(len(bins)-1), columns=["bin", "min(inclusive)", "max(exclusive)", "new_obs", "old_obs", "old_obs_perc", "new_obs_perc", "PSI"])

        for i in range(len(bins)-1):
            bin_min = bins[i]
            bin_max = bins[i+1]
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("bin")] = f"{bin_min}-{bin_max}"
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("min(inclusive)")] = bin_min
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("max(exclusive)")] = bin_max
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("old_obs")] = ((pred_score["y_predict"] >= bin_min) & (pred_score["y_predict"] < bin_max)).sum()
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("new_obs")] = ((pred_score_new["y_predict"] >= bin_min) & (pred_score_new["y_predict"] < bin_max)).sum()
        
        total_old = obs_new_vs_old.iloc[:, obs_new_vs_old.columns.get_loc("old_obs")].sum()
        total_new = obs_new_vs_old.iloc[:, obs_new_vs_old.columns.get_loc("new_obs")].sum()

        for i in range(len(bins)-1):
            old_obs_i = obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("old_obs")]
            new_obs_i = obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("new_obs")]
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("old_obs_perc")] = (old_obs_i / total_old) * 100 if total_old != 0 else 0
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("new_obs_perc")] = (new_obs_i / total_new) * 100 if total_new != 0 else 0
            psi_value = (old_obs_i / total_old - new_obs_i / total_new) * math.log((old_obs_i / total_old) / (new_obs_i / total_new)) if new_obs_i != 0 else 0
            obs_new_vs_old.iloc[i, obs_new_vs_old.columns.get_loc("PSI")] = psi_value

        # print(obs_new_vs_old)

        psi_values = obs_new_vs_old[['bin', 'new_obs', 'new_obs_perc', 'old_obs', 'old_obs_perc', 'PSI']]
        psi_values.to_csv(path2 + 'mrdc_model_drift/psi_value/psi_'+ month + ".csv")

    return obs_new_vs_old[['bin', 'new_obs', 'new_obs_perc', 'old_obs', 'old_obs_perc', 'PSI']]


In [ ]:
months = ['sept_23', 'oct_23', 'nov_23', 'dec_23', 'jan_24']
for i in range(len(months)): 
    df = load_dataset(months[i])
    df = preprocess(df)
    psi_value = calculate_psi(df_ref, df, ['predictions'], months[i] )
    print("-"*75)
    print(psi_value,"\n\n")

        bin  new_obs  new_obs_perc   old_obs  old_obs_perc       PSI
0     0-100   7965.0     45.242829  109620.0     32.588047  0.041520
1   100-200   5188.0     29.468901  103327.0     30.717252  0.000518
2   200-300   1886.0     10.712866   50773.0     15.093897  0.015020
3   300-500   1349.0      7.662596   39115.0     11.628184  0.016540
4   500-700    388.0      2.203919   14031.0      4.171163  0.012550
5   700-800    264.0      1.499574    5843.0      1.737018  0.000349
6   800-850    162.0      0.920193    3240.0      0.963194  0.000020
7   850-900    125.0      0.710026    2939.0      0.873712  0.000340
8   900-950    157.0      0.891792    3854.0      1.145725  0.000636
9  950-1000    121.0      0.687305    3639.0      1.081809  0.001790 


        bin  new_obs  new_obs_perc   old_obs  old_obs_perc       PSI
0     0-100  14320.0     45.484865  109620.0     32.588047  0.043002
1   100-200   9402.0     29.863736  103327.0     30.717252  0.000241
2   200-300   3281.0     10.421